# End-to-End NED Tutorial

In this tutorial, we walk through how to use Bootleg as an end-to-end pipeline to detect and label entities in a set of sentences. First, we show how to use Bootleg to detect and disambiguate mentions to entities. We then compare to an existing system named TAGME. 

This tutorial assumes you want to use Bootleg on full datasets. You can also use Bootleg in annotator mode:

```
pip install bootleg
from bootleg.end2end.bootleg_annotator import BootlegAnnotator
ann = BootlegAnnotator()
ann.label_mentions("Bob Dylan release Desire")["titles"]
```

To understand how Bootleg performs on more natural language than we find in Wikipedia, we hand label the mentions and corresponding entities in 50 questions sampled from the [Natural Questions dataset (Google)](https://ai.google.com/research/NaturalQuestions). We will evaluate our *uncased* Bootleg model. However, we have manually cased the data in case you want to try our cased model instead.

### Requirements

You will need to download the following files for this notebook:
- Pretrained Bootleg uncased model and config [here](https://bootleg-data.s3-us-west-2.amazonaws.com/models/lateset/bootleg_uncased.tar.gz). Cased model and config [here](https://bootleg-data.s3-us-west-2.amazonaws.com/models/lateset/bootleg_cased.tar.gz)
- Sample of Natural Questions with hand-labelled entities [here](https://bootleg-data.s3-us-west-2.amazonaws.com/data/lateset/nq.tar.gz)
- Entity data [here](https://bootleg-data.s3-us-west-2.amazonaws.com/data/lateset/entity_db.tar.gz)

For convenience, you can run the commands below (from the root directory of the repo) to download all the above files and unpack them to `models` and `data` directories. It will take several minutes to download all the files. 

```
    # use cased for cased model
    bash tutorials/download_model.sh uncased
    bash tutorials/download_data.sh
```

You can also run directly in this notebook by

In [ ]:
!sh download_model.sh uncased
!sh download_data.sh

In [1]:
from pathlib import Path
import pandas as pd

# set up logging
import sys
import logging
from importlib import reload
reload(logging)
# Set to logging.DEBUG for more logging output
logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


# root_dir = FILL IN FULL PATH TO DIRECTORY WHERE DATA IS DOWNLOADED (e.g., root_dir/data and root_dir/models)
root_dir = Path("tutorial_data")
cand_map = root_dir / 'data/entity_db/entity_mappings/alias2qids.json'

If you have a GPU with at least 12GB of memory available, set the below to 0 to run inference on a GPU. 

In [2]:
device = -1

## 1. Detect Mentions
Bootleg uses a simple mention extraction algorithm that extracts mentions using a given candidate map. We will use a Wikipedia candidate map that we mined using Wikipedia anchor links and Wikidata aliases for a total of ~15 million mentions (provided in the Requirements section of this notebook).

For the input dataset for the end-to-end pipeline, we assume a jsonlines file with a single dictionary with the key "sentence" and value as the text of the sentence, per line. For instance, you may have a file with the lines:

    {"sentence": "Who did the voice of the magician in Frosty the Snowman"}
    {"sentence": "What is considered the Outer Banks in North Carolina"}
    
Below, we have additional keys to keep track of the hand-labelled mentions, but this is purely for evaluating the quality of the end-to-end pipeline and is not needed in the common use cases of using Bootleg to detect and label mentions.

In [3]:
nq_sample_orig = root_dir / 'data/nq/test_50.jsonl'
nq_sample_bootleg = root_dir / 'data/nq/test_50_bootleg.jsonl'

In [4]:
from bootleg.end2end.extract_mentions import extract_mentions
verbose = False
extract_mentions(in_filepath=nq_sample_orig, out_filepath=nq_sample_bootleg, cand_map_file=cand_map, verbose=verbose)

By looking at a sample of the extracted mentions, we can compare the mention extraction phase to the hand-labelled mentions.

In [5]:
from utils import load_mentions

orig_mentions_df = load_mentions(nq_sample_orig)
bootleg_mentions_df = load_mentions(nq_sample_bootleg)

# join dataframes and sample
res = pd.merge(orig_mentions_df, bootleg_mentions_df, on=['sentence'], suffixes=['_hand', '_bootleg'])
display(res.head(15))

,sentence,aliases_hand,spans_hand,aliases_bootleg,spans_bootleg
0,Who did the voice of the magician in Frosty the Snowman,[frosty the snowman],"[[8, 11]]","[voice of, magician, frosty the snowman]","[[3, 5], [6, 7], [8, 11]]"
1,What is considered the Outer Banks in North Carolina,"[outer banks, north carolina]","[[4, 6], [7, 9]]","[outer banks, north carolina]","[[4, 6], [7, 9]]"
2,The Nashville sound brought a polished and cosmopolitan sound to country music by,"[nashville sound, country music]","[[1, 3], [10, 12]]","[nashville sound, music by]","[[1, 3], [11, 13]]"
3,What channel is the Premier League on in France,"[premier league, france]","[[4, 6], [8, 9]]","[premier league, france]","[[4, 6], [8, 9]]"
4,I Love It ( feat . Charli XCX ) Icona Pop,"[i love it, charli xcx, icona pop]","[[0, 3], [6, 8], [9, 11]]","[charli xcx, icona pop]","[[6, 8], [9, 11]]"
5,The U.S. Supreme Court hears appeals from circuit courts,"[u.s. supreme court, circuit courts]","[[1, 4], [7, 9]]","[us supreme court, circuit courts]","[[1, 4], [7, 9]]"
6,Why does the author say that the vampire in Nosferatu is named Count Orlok and not Count Dracula,"[nosferatu, count orlok, count dracula]","[[9, 10], [12, 14], [16, 18]]","[vampire, nosferatu, count orlok, count dracula]","[[7, 8], [9, 10], [12, 14], [16, 18]]"
7,Is there an active volcano in New Zealand,[new zealand],"[[6, 8]]","[volcano, new zealand]","[[4, 5], [6, 8]]"
8,Once Upon a Time Season 6 episode list,[once upon a time season 6],"[[0, 6]]","[upon a time, season 6, episode list]","[[1, 4], [4, 6], [6, 8]]"
9,Who is the former co-chairman Goldman Sachs who became a U.S. Secretary of the Treasury,"[goldman sachs, us secretary of the treasury]","[[5, 7], [10, 15]]","[goldman sachs, us secretary of the treasury]","[[5, 7], [10, 15]]"


In the sample above, we see that generally Bootleg detects the same mentions as the hand-labelled mentions, however sometimes Bootleg extracts extra mentions (e.g "colonies" in "Where did Britain create colonies for its empire"). This is expected as we would rather the mention detection step filter out too few mentions than too many. It will be the job of the backbone model and postprocessing to filter out these extra mentions, by either thresholding the prediction probability or predicting a candidate that represents "No Candidate" (we refer to this as "NC").  

## 2. Disambiguate Mentions to Entities

We run inference using a pretrained Bootleg model to disambiguate the extracted mentions to Wikidata QIDs. 

First, load the model config so we can set additional parameters and load the saved model during evaluation. We need to update the config parameters to point to the downloaded model checkpoint and data.

In [6]:
from bootleg.utils.parser.parser_utils import parse_boot_and_emm_args
from bootleg.utils.utils import load_yaml_file
from bootleg.run import run_model

config_in_path = root_dir / 'models/bootleg_uncased/bootleg_config.yaml'

config_args = load_yaml_file(config_in_path)

# decrease number of data threads as this is a small file
config_args["run_config"]["dataset_threads"] = 2
config_args["run_config"]["log_level"] = "info"
# set the model checkpoint path 
config_args["emmental"]["model_path"] = str(root_dir / 'models/bootleg_uncased/bootleg_wiki.pth')

# set the path for the entity db and candidate map
config_args["data_config"]["entity_dir"] = str(root_dir / 'data/entity_db')
config_args["data_config"]["alias_cand_map"] = "alias2qids.json"

# set the data path and kore50 test file 
config_args["data_config"]["data_dir"] = str(root_dir / 'data/nq')

# to speed things up for the tutorial, we have already prepped the data with the mentions detected by Bootleg
config_args["data_config"]["test_dataset"]["file"] = nq_sample_bootleg.name

# set the embedding paths 
config_args["data_config"]["emb_dir"] =  str(root_dir / 'data/entity_db')
config_args["data_config"]["word_embedding"]["cache_dir"] =  str(root_dir / 'data/pretrained_bert_models')

# set the devie if on CPU
config_args["emmental"]["device"] = device

# save the new args (helps if you want to run things via command line)
config_args = parse_boot_and_emm_args(config_args)

In [7]:
bootleg_label_file, _ = run_model(mode="dump_preds", config=config_args)

2021-03-17 15:34:16,174 Setting logging directory to: bootleg_logs/wiki_full_ft/2021_03_17/15_34_15/2d3cede7
2021-03-17 15:34:16,359 Loading Emmental default config from /dfs/scratch0/lorr1/env_bootleg_38/lib/python3.8/site-packages/emmental/emmental-default-config.yaml.
2021-03-17 15:34:16,361 Updating Emmental config from user provided config.
2021-03-17 15:34:16,362 Set random seed to 1234.
2021-03-17 15:34:17,406 COMMAND: /dfs/scratch0/lorr1/env_bootleg_38/lib/python3.8/site-packages/ipykernel_launcher.py -f /dfs/scratch0/lorr1/projects/:/afs/cs.stanford.edu/u/lorr1/.local/apt-cache/share/jupyter/runtime/kernel-3edac498-7b5f-4175-b525-87fb3287db3f.json
2021-03-17 15:34:17,407 Saving config to bootleg_logs/wiki_full_ft/2021_03_17/15_34_15/2d3cede7/parsed_config.yaml
2021-03-17 15:34:17,738 Git Hash: Not able to retrieve git hash
2021-03-17 15:34:17,740 Loading entity symbols...
2021-03-17 15:37:51,544 Starting to build data for test from ../tutorial_data/data/nq/test_50_bootleg.json

/dfs/scratch0/lorr1/projects/bootleg/bootleg/datasets/dataset.py:1020: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  guid_dtype = np.dtype(
/dfs/scratch0/lorr1/projects/bootleg/bootleg/layers/alias_to_ent_encoder.py:98: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  alias2entity_table = torch.from_numpy(alias2entity_table)
Reading in ../tutorial_data/data/nq/prep/prep_test_dataset_files/create_examples_input/out_0.jsonl: 100%|██████████| 25/25 [00:00<00:00, 399.17it/

2021-03-17 15:40:17,446 Loading data from ../tutorial_data/data/nq/prep/test_50_bootleg_bert-base-uncased_L100_A10_InC1_Aug1/ned_data.bin and ../tutorial_data/data/nq/prep/test_50_bootleg_bert-base-uncased_L100_A10_InC1_Aug1/ned_label.bin
2021-03-17 15:40:17,449 Building type labels from scatch.
2021-03-17 15:43:03,786 Creating type prediction labeled data using 2 threads


Verifying type labels: 100%|██████████| 49/49 [00:00<00:00, 21994.96it/s]


2021-03-17 15:43:15,310 Final data initialization time for test is 323.7640495300293s
2021-03-17 15:43:15,789 Built dataloader for test set with 49 and 1 threads samples (Shuffle=False, Batch size=32).
2021-03-17 15:43:15,837 Building slice dataset for test from ../tutorial_data/data/nq/test_50_bootleg.jsonl.
2021-03-17 15:43:15,882 Building dataset from scratch. Saving to ../tutorial_data/data/nq/prep/test_50_bootleg_bert-base-uncased_L100_A10_InC1_Aug1
2021-03-17 15:43:15,884 Strating to extract examples with 2 threads


...

Building sent idx to row idx mapping: 100%|██████████| 50/50 [00:00<00:00, 11491.24it/s]

2021-03-17 15:43:17,766 Final slice data initialization time from test is 1.9294226169586182s
2021-03-17 15:43:17,767 Updating Emmental config from user provided config.
2021-03-17 15:43:17,768 Set random seed to 1234.
2021-03-17 15:43:17,778 Starting Bootleg Model
2021-03-17 15:43:17,779 Created emmental model Bootleg that contains task set().


2021-03-17 15:43:23,390 Loading embeddings...
2021-03-17 15:46:59,266 Created task: NED
2021-03-17 15:46:59,268 Moving bert module to CPU.
2021-03-17 15:46:59,279 Moving embedding_payload module to CPU.
2021-03-17 15:46:59,281 Moving attn_network module to CPU.
2021-03-17 15:46:59,286 Moving pred_layer module to CPU.
2021-03-17 15:46:59,287 Moving learned module to CPU.
2021-03-17 15:46:59,287 Moving title_static module to CPU.
2021-03-17 15:46:59,288 Moving learned_type module to CPU.
2021-03-17 15:46:59,289 Moving learned_type_wiki module to CPU.
2021-03-17 15:46:59,290 Moving learned_type_relations module to CPU.
2021-03-17 15:46:59,291 Moving adj_index module to CPU.
2021-03-17 15:47:03,644 Created task: Type
2021-03-17 15:47:03,647 Moving bert module to CPU.
2021-03-17 15:47:03,651 Moving embedding_payload module to CPU.
2021-03-17 15:47:03,652 Moving attn_network module to CPU.
2021-03-17 15:47:03,655 Moving pred_layer module to CPU.
2021-03-17 15:47:03,655 Moving learned module 

Evaluating Bootleg (test): 100%|██████████| 2/2 [00:08<00:00,  4.02s/it]


2021-03-17 15:48:01,694 Finished dumping. Merging results across accumulation steps.
2021-03-17 15:48:01,735 Bootleg labels saved at bootleg_logs/wiki_full_ft/2021_03_17/15_34_15/2d3cede7/test_50_bootleg/bootleg_wiki/bootleg_labels.jsonl


**Note that Bootleg automatically handles prepping of new data files for running. These are all saved in `data_config.data_dir`/`data_config.data_prep_dir`. If you change the contents of the underlying `jsonl` file _without_ removing the saved prep file or setting `data_config.overwrite_preprocessed_data` to be `True`, Bootleg will reuse the old prepped file.**


We can now evaluate the overall quality of the end-to-end pipeline via precision / recall metrics, where the *recall* indicates what proportion of the hand-labelled mentions Bootleg correctly detects and disambiguates, and *precision* indicates what proportion of the mentions that Bootleg labels are correct. For instance, if Bootleg only labelled the few mentions it was very confident in, then it would have a low recall and high precision.

To detect if mentions match the hand-labelled mention spans, we report weak and exact match metrics. Weak means the predicted and gold span boundaries just need to overlap for an entity (e.g., predicted mention 'the wizard of oz' is counted as correct for the gold mention 'wizard of oz' if the correct entity is predicted). 

In [8]:
from utils import compute_metrics
bootleg_end2end_errors = compute_metrics(gold_file=nq_sample_orig,       
                                 pred_file=bootleg_label_file, 
                                 threshold=0.5)

WEAK MATCHING
precision = 61 / 80 = 0.7625
recall = 61 / 78 = 0.782051282051282
f1 = 0.7721518987341772

EXACT MATCHING
precision = 61 / 80 = 0.7625
recall = 61 / 78 = 0.782051282051282
f1 = 0.7721518987341772


We can examine errors in the end-to-end pipeline below. As you increase the threshold in the `compute_metrics` command, entities with a prediction probability less than the threshold will be filtered out. If too few entities are predicted, lowering the threshold may help.  

In [42]:
pd.DataFrame(bootleg_end2end_errors).sample(10)

,sent_idx,text,gold_aliases,gold_qids,gold_spans,pred_aliases,pred_qids,pred_spans,pred_probs
14,4,I Love It ( feat . Charli XCX ) Icona Pop,"[i love it, charli xcx, icona pop]","[Q3273659, Q5084390, Q808703]","[[0, 3], [6, 8], [9, 11]]","[charli xcx, icona pop]","[Q5084390, Q808703]","[[5, 9], [9, 11]]","[1.0, 0.9873091578]"
7,42,Who proposed the coordinate system to describe the position of a point in a plane accurately,[coordinate system],[Q62912],"[[3, 5]]",[coordinate system],[Q11210],"[[3, 5]]",[1.0]
5,37,Landmark Supreme Court cases dealing with the First Amendment,"[supreme court, first amendment]","[Q11201, Q12616]","[[1, 3], [7, 9]]","[supreme court cases, first amendment]","[Q6646863, Q12616]","[[1, 4], [7, 9]]","[1.0, 0.9999812841]"
18,8,Once Upon a Time Season 6 episode list,[once upon a time season 6],[Q23301616],"[[0, 6]]","[season 6, episode list]",[Q2404330],"[[4, 6]]",[0.5048642159]
22,13,Where was 10 Things I Hate About You filmed school,[10 things i hate about you],[Q169082],"[[2, 8]]",[10 things i hate about you],[Q169074],"[[2, 8]]",[0.5601187348]
1,31,Who does Oregon state play in the College World Series,"[oregon state, college world series]","[Q7101349, Q787505]","[[2, 4], [7, 10]]","[oregon state, college world series]",[Q787505],"[[7, 10]]",[0.8851752281]
16,6,Why does the author say that the vampire in Nosferatu is named Count Orlok and not Count Dracula,"[nosferatu, count orlok, count dracula]","[Q151895, Q1442062, Q3266236]","[[9, 10], [12, 14], [16, 18]]","[vampire, nosferatu, count orlok, count dracula]","[Q7912955, Q151895, Q1442062, Q3266236]","[[7, 8], [9, 10], [12, 14], [16, 18]]","[0.7404002547, 0.9794661999, 1.0, 0.9972344041]"
6,40,Where does the last name Vigil come from,[vigil],[Q16878937],"[[5, 6]]",[vigil],[],[],[]
3,35,Reasons why South Africa should include renewable energy in its energy mix,"[south africa, renewable energy]","[Q258, Q12705]","[[2, 4], [6, 8]]","[reasons why, south africa, renewable energy, energy mix]","[Q7028249, Q258, Q12705, Q1341346]","[[0, 2], [2, 4], [6, 8], [10, 12]]","[1.0, 0.999255836, 0.8151838183, 1.0]"
10,48,What was the Japanese motivation for bombing Pearl Harbor,"[japanese, pearl harbor]","[Q188712, Q127091]","[[3, 4], [7, 9]]","[motivation, pearl harbor]","[Q644302, Q127091]","[[4, 5], [7, 9]]","[0.9905021191, 0.757401526]"


Some of the errors Bootleg makes is predicting too general of a candidate (e.g. Oregon State Beavers instead of Oregon State Beavers baseball). Other errors are due to ambiguous sentences (e.g. "cast of characters in fiddler on the roof" -> should this be the movie or the musical?). Finally another bucket of errors suggests that we need to boost certain training signals -- this is an area we're actively pursuing in Bootleg with an investigation of model guidability!

## 3. Compare to TAGME 

To get a sense of how Bootleg is doing compared to other systems, we evaluate [TAGME](https://arxiv.org/pdf/1006.3498.pdf), an existing tool to extract and disambiguate mentions. To run TAGME, you need to get a (free) authorization token. Instructions for obtaining a token are [here](https://sobigdata.d4science.org/web/tagme/tagme-help). You will need to verify your account and then follow the "access the VRE") link. We've also provided the file with TAGME labels for a given threshold for download if you want to skip the authorization token.

We note that unlike TAGME, Bootleg also outputs contextual entity embeddings which can be loaded for use in downstream tasks (e.g. relation extraction, question answering). Check out the Entity Embedding tutorial for more details! 

In [47]:
import tagme
# Set the authorization token for subsequent calls.
tagme.GCUBE_TOKEN = ""

In [13]:
tagme_label_file = root_dir / '/data/nq/test_50_tagme.jsonl'

If you do not have a token, skip the cell below and load the pre-generated TAGME labels. If you do have a token, you can play with changing the threshold below and see how it affects the results. Increasing the threshold increases the precision but decreases the recall as TAGME, as TAGME will label fewer mentions.

In [63]:
from utils import tagme_annotate
# As the threshold increases, the precision increases, but the recall decreases
tagme_annotate(in_file=nq_sample_orig, out_file=tagme_label_file, threshold=0.2)

No wikidata id found for Frosty the Snowman (film)
No wikidata id found for The Bachelor (U.S. TV series)
No wikidata id found for House of Cards (U.S. TV series)


Note that we do not set the threshold here when computing metrics for TAGME as TAGME predictions are already thresholded in the `tagme_annotate` function. 

In [64]:
tagme_errors = compute_metrics(gold_file=nq_sample_orig, pred_file=tagme_label_file)

WEAK MATCHING
precision = 53 / 101 = 0.5247524752475248
recall = 53 / 78 = 0.6794871794871795
f1 = 0.5921787709497207

EXACT MATCHING
precision = 52 / 101 = 0.5148514851485149
recall = 52 / 78 = 0.6666666666666666
f1 = 0.5810055865921788


Several Wikidata ids are not recovered due to out of date Wikipedia titles in TAGME predictions. Even when including these three samples as correct predictions, we see that Bootleg is able to outperform TAGME in both precision and recall.